In [1]:
import sys
import importlib

In [2]:
from saxstats import saxstats
from Ligand import Ligand
from WAXS import *
from PDB import PDB
import rdkit
from rdkit import Chem
import numpy as np
from Geometry import *

In [3]:
pro = PDB('1FIN_apo.pdb')

This PDB does not seem to have hydrogens modeled


In [4]:
print(np.sum([True for x in pro.elements if x == 'H']))
print(np.sum([True for x in pro.elements if x != 'H']))

0.0
2398


In [18]:
%%time
#lig = Ligand('c1ccccc1CCC')
lig = Ligand('1FIN_ligand.pdb') 
lig.generate_conformers(5)
# Chem.MolToPDBFile(lig.molecule, 'test.pdb')
# print(lig.get_coordinates(0))

Generated 5 conformers
CPU times: user 40.5 ms, sys: 2.82 ms, total: 43.3 ms
Wall time: 41.1 ms


RDKit WARNING: [19:07:33] Molecule does not have explicit Hs. Consider calling AddHs()
[19:07:33] Molecule does not have explicit Hs. Consider calling AddHs()
RDKit WARNING: [19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 0
RDKit WARNING: [19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 8
RDKit WARNING: [19:07:33] UFFTYPER: Warning: hybridization set to SP3 for atom 8


In [6]:
import time

In [7]:
pocket = PDB('test_fpocket/1FIN_apo_out/1FIN_apo_out.pdb')

This PDB does not seem to have hydrogens modeled


In [23]:
t0 = time.time()
confID = np.random.randint(0, 10)
print(f'Trying conformer ID {confID}')
pv, pg, ps, lv, lg, ls = overlap_grid(pro, lig, conformerID=0, pocket=pocket, 
                                      grid_spacing=1.0, radius=1.0, write_grid=False, timing=True)
t1 = time.time()
print(f'Time elapsed: {t1-t0:.3f} s')
# saxstats.write_mrc(pv, ps, 'test_protein_convolved.mrc')
# saxstats.write_mrc(lv, ls, 'test_ligand.mrc')

Trying conformer ID 4
0.008 ms protein grid generation
4.260 ms ligand grid generation
41.020 ms binary dilation
There are 12338 points deemed possible out of 253825 points purely from overlap.
0.867 ms remove far zone
0.219 ms pocket distance calculation
There are 63 points deemed possible out of 253825 points after pocket overlap.
0.596 ms pocket filter
0.016 ms write out grid points
Time elapsed: 0.048 s


In [19]:
rmat = rotation_sampling(8)
num_conditions = 5 * len(rmat)
t0 = time.time()
t1 = time.time()
condition_idx = 0
XS_calc_num = 0
for confID in range(5):
    for idx, rot in enumerate(rmat):
        if idx > 0 and idx % 5 == 0:
            print(f'Conformation {confID}, orientation {idx}')
        pv, pg, ps, lv, lg, ls = overlap_grid(pro, lig, conformerID=confID, rotation=rot, pocket=pocket, 
                                              grid_spacing=1.0, radius=1.0, write_grid=False, timing=False, 
                                              printing=False)
        XS_calc_num += np.sum(pv)
        condition_idx += 1
        t2 = time.time()
        if (t2-t1) > 5:
            print(f'Elapsed: {t2-t0:.3f} s. Estimated remaining: {(num_conditions - condition_idx) * (t2-t0) / condition_idx}')
            t1 = t2

Conformation 0, orientation 5
Conformation 0, orientation 10
Conformation 0, orientation 15
Conformation 0, orientation 20
Conformation 0, orientation 25
Conformation 0, orientation 30
Conformation 0, orientation 35
Conformation 0, orientation 40
Conformation 0, orientation 45
Conformation 0, orientation 50
Conformation 0, orientation 55
Conformation 0, orientation 60
Conformation 0, orientation 65
Conformation 0, orientation 70
Conformation 0, orientation 75
Conformation 0, orientation 80
Conformation 0, orientation 85
Conformation 0, orientation 90
Conformation 0, orientation 95
Conformation 0, orientation 100
Conformation 0, orientation 105
Conformation 0, orientation 110
Conformation 0, orientation 115
Elapsed: 5.007 s. Estimated remaining: 103.62314551967685
Conformation 0, orientation 120
Conformation 0, orientation 125
Conformation 0, orientation 130
Conformation 0, orientation 135
Conformation 0, orientation 140
Conformation 0, orientation 145
Conformation 0, orientation 150
Co

Conformation 2, orientation 180
Conformation 2, orientation 185
Conformation 2, orientation 190
Conformation 2, orientation 195
Conformation 2, orientation 200
Conformation 2, orientation 205
Conformation 2, orientation 210
Conformation 2, orientation 215
Conformation 2, orientation 220
Conformation 2, orientation 225
Conformation 2, orientation 230
Conformation 2, orientation 235
Conformation 2, orientation 240
Conformation 2, orientation 245
Conformation 2, orientation 250
Conformation 2, orientation 255
Conformation 2, orientation 260
Conformation 2, orientation 265
Conformation 2, orientation 270
Conformation 2, orientation 275
Elapsed: 60.270 s. Estimated remaining: 58.323991576677834
Conformation 2, orientation 280
Conformation 2, orientation 285
Conformation 2, orientation 290
Conformation 2, orientation 295
Conformation 2, orientation 300
Conformation 2, orientation 305
Conformation 2, orientation 310
Conformation 2, orientation 315
Conformation 2, orientation 320
Conformation 

Conformation 4, orientation 350
Conformation 4, orientation 355
Conformation 4, orientation 360
Conformation 4, orientation 365
Conformation 4, orientation 370
Conformation 4, orientation 375
Conformation 4, orientation 380
Conformation 4, orientation 385
Conformation 4, orientation 390
Conformation 4, orientation 395
Conformation 4, orientation 400
Conformation 4, orientation 405
Conformation 4, orientation 410
Elapsed: 115.549 s. Estimated remaining: 4.697114839786437
Conformation 4, orientation 415
Conformation 4, orientation 420
Conformation 4, orientation 425
Conformation 4, orientation 430
Conformation 4, orientation 435
Conformation 4, orientation 440
Conformation 4, orientation 445
Conformation 4, orientation 450
Conformation 4, orientation 455
Conformation 4, orientation 460
Conformation 4, orientation 465
Conformation 4, orientation 470
Conformation 4, orientation 475
Conformation 4, orientation 480
Conformation 4, orientation 485
Conformation 4, orientation 490
Conformation 

In [20]:
XS_calc_num

131816

In [23]:
pg[pv.flatten()]

array([[-13.,  -7.,  15.],
       [ -7.,  -7.,  16.],
       [ -6.,  -6.,  16.],
       [ -1.,  -2., -21.],
       [ -1.,  -1., -21.],
       [ -1.,   0., -21.],
       [  0.,  -2., -21.],
       [  0.,  -1., -21.],
       [  0.,  -1., -20.],
       [  0.,   0., -21.],
       [  1.,  -2., -21.],
       [  1.,  -1., -21.],
       [  1.,  -1., -20.],
       [  1.,   0., -21.],
       [  2.,  -2., -20.],
       [  2.,  -1., -20.],
       [  3.,  -2., -20.],
       [  4.,  14., -19.],
       [  5.,  13., -19.],
       [  5.,  14., -19.],
       [  5.,  15., -18.],
       [  6.,  14., -18.],
       [  6.,  15., -18.],
       [  9.,  -9.,  -5.],
       [ 10.,  -9.,  -6.],
       [ 10.,  -9.,  -5.],
       [ 10.,  -8.,  -4.],
       [ 11.,  -9.,  -7.],
       [ 11.,  -9.,  -6.],
       [ 11.,  -9.,  -5.],
       [ 11.,  -8.,  -8.],
       [ 11.,  -8.,  -7.],
       [ 11.,  -8.,  -6.],
       [ 11.,  -8.,  -5.],
       [ 11.,  -8.,  -4.],
       [ 12.,  -9.,  -7.],
       [ 12.,  -9.,  -6.],
 

In [9]:
lig.elements

array(['P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'O',
       'C', 'C', 'O', 'C', 'O', 'C', 'O', 'C', 'N', 'C', 'N', 'C', 'C',
       'N', 'N', 'C', 'N', 'C'], dtype='<U1')

In [11]:
def write_grid_search_xyz(lig, gp, fname='grid_ligand.xyz'):
    lig_coord_init = lig.get_coordinates(0)
    lig_coord_origin = lig_coord_init - np.mean(lig_coord_init, axis=0) #- np.mean(lig_coord_init, axis=0)
    with open(fname, 'w') as f:
        for idx, gp_xyz in enumerate(gp):
            lig_coord_this_grid = lig_coord_origin + gp_xyz
            f.write(f'{lig.num_atoms+1}\n')
            f.write(f'Grid pt {idx}: {gp_xyz[0]:.2f} {gp_xyz[1]:.2f} {gp_xyz[2]:.2f}\n')
            for ele, xyz in zip(lig.elements, lig_coord_this_grid):
                f.write(f'{ele} {xyz[0]} {xyz[1]} {xyz[2]}\n')
            f.write(f'X {gp_xyz[0]} {gp_xyz[1]} {gp_xyz[2]}\n')
        f.write('\n\n')

In [12]:
write_grid_search_xyz(lig, pg[pv.flatten()], fname='grid_ligand3.xyz')

In [13]:
write_grid_search_xyz(lig, [lig.get_coordinates(0).mean(0)], fname='ligand_COM.xyz')

In [14]:
lig.get_coordinates(0).mean(0)

array([ 7.56209677, -9.4603871 , -2.96754839])

In [15]:
np.stack(min_max(lig.get_coordinates(0))).mean(0)

array([ 7.108 , -9.8465, -3.143 ])

In [16]:
pocket = PDB('test_fpocket/1FIN_apo_out/1FIN_apo_out.pdb')

This PDB does not seem to have hydrogens modeled


In [16]:
pocket_coord = pocket.pdb.coords[pocket.elements == 'Ve']

In [17]:
pocket_coord

array([[13.756, -8.239, -7.681],
       [13.826, -8.157, -7.646],
       [13.726, -8.139, -7.627],
       ...,
       [17.881,  8.429, -5.555],
       [17.822,  8.284, -5.585],
       [17.664,  8.338, -4.666]])

In [18]:
pg.shape

(253825, 3)

In [19]:
pv2 = distance_matrix(pg[pv.flatten()], pocket_coord).min(1) < 2

In [20]:
write_grid_points(pg[pv.flatten()], pv2, fname='grid3.pdb')

In [21]:
pg[pv.flatten()][pv2].shape

(99, 3)

In [8]:
np.arange(3)

array([0, 1, 2])

In [50]:
def raster_unit_sphere(num=1000):
    L = np.sqrt(num * np.pi);
    pt = []
    for i in range(num):
        h = 1.0 - (2.0 * i + 1.0) / num
        p = np.arccos(h)
        t = L * p
        xu = np.sin(p) * np.cos(t)
        yu = np.sin(p) * np.sin(t)
        zu = np.cos(p)
        pt.append([xu, yu, zu])

    return np.array(pt)


In [51]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    if any(v): #if not all zeros then 
        c = np.dot(a, b)
        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return np.eye(3) #cross of all zeros only occurs on identical directions


In [52]:
us = raster_unit_sphere()

In [53]:
zvec = np.array([0,0,1])

In [54]:
rmat = np.array([rotation_matrix_from_vectors(zvec, x) for x in us])

In [55]:
(lig.get_coordinates(0) @ rmat[0]).shape

(31, 3)

In [56]:
def write_rotate_ligand_xyz(lig, rmat, us, fname='rotate_ligand.xyz'):
    lig_coord_init = lig.get_coordinates(0)
    lig_coord_origin = lig_coord_init - np.mean(lig_coord_init, axis=0) #- np.mean(lig_coord_init, axis=0)
    with open(fname, 'w') as f:
        for idx, (r, u) in enumerate(zip(rmat, us)):
            lig_coord_this_grid = r @ lig_coord_origin 
            f.write(f'{lig.num_atoms+1}\n')
            f.write(f'Rotation pt {idx}: {u[0]:.2f} {u[1]:.2f} {u[2]:.2f}\n')
            for ele, xyz in zip(lig.elements, lig_coord_this_grid):
                f.write(f'{ele} {xyz[0]} {xyz[1]} {xyz[2]}\n')
            f.write(f'X {u[0]*5} {u[1]*5} {u[2]*5}\n')
        f.write('\n\n')

In [59]:
def rotz(deg):
    rad = deg / 180 * np.pi
    return np.array([[np.cos(rad), np.sin(rad), 0], [-np.sin(rad), np.cos(rad), 0], [0, 0, 1]])
def rotx(deg):
    rad = deg / 180 * np.pi
    return np.array([[1, 0, 0], [0, np.cos(rad), np.sin(rad)], [0, -np.sin(rad), np.cos(rad)]])
def roty(deg):
    rad = deg / 180 * np.pi
    return np.array([[np.cos(rad), 0, -np.sin(rad)], [0, 1, 0], [np.sin(rad), 0, np.cos(rad)]])

In [68]:
rmat = []
sampling = 8
for i in range(0, 360, 360//sampling):
    for j in range(0, 360, 360//sampling):
        for k in range(0, 360, 360//sampling):
            rmat.append(rotx(i) @ roty(j) @ rotz(k))
rmat = np.array(rmat)

In [70]:
rmat.shape

(512, 3, 3)

In [71]:
us = rmat @ zvec

In [72]:
us.shape

(512, 3)

In [73]:
write_rotate_ligand_xyz(lig, rmat, us)